# Examine covid-19 variants

In [95]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv('original/variant_list.tsv.gz', sep="\t", low_memory=False)
print("%s rows x %s columns" % (data.shape[0], data.shape[1]))
data.head()

77121 rows x 14 columns


,Sample,CHROM,POS,REF,ALT,DP,AF,SB,DP4,EFF[*].IMPACT,EFF[*].FUNCLASS,EFF[*].EFFECT,EFF[*].GENE,EFF[*].CODON
0,SRR10903401,NC_045512,1409,C,T,126,0.039683,1,"67,54,2,3",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,orf1ab,Cat/Tat
1,SRR10903401,NC_045512,1821,G,A,93,0.096774,0,"48,36,5,4",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,orf1ab,gGt/gAt
2,SRR10903401,NC_045512,1895,G,A,106,0.037736,0,"51,51,2,2",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,orf1ab,Gta/Ata
3,SRR10903401,NC_045512,2407,G,T,123,0.024390,0,"57,63,1,2",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,orf1ab,aaG/aaT
4,SRR10903401,NC_045512,3379,A,G,121,0.024793,0,"56,62,1,2",LOW,SILENT,SYNONYMOUS_CODING,orf1ab,gtA/gtG


Great. Let's first create a new column with variant identifier to be used for later mining.

In [81]:
data['transaction_id'] = data.apply(
    lambda row: row.CHROM + ':' + row.REF + ':' + str(row.POS) + ':' + row.ALT, axis=1)
data

,Sample,CHROM,POS,REF,ALT,DP,AF,SB,DP4,EFF[*].IMPACT,EFF[*].FUNCLASS,EFF[*].EFFECT,EFF[*].GENE,EFF[*].CODON,transaction_id
0,SRR10903401,NC_045512,1409,C,T,126,0.039683,1,"67,54,2,3",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,orf1ab,Cat/Tat,NC_045512:C:1409:T
1,SRR10903401,NC_045512,1821,G,A,93,0.096774,0,"48,36,5,4",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,orf1ab,gGt/gAt,NC_045512:G:1821:A
2,SRR10903401,NC_045512,1895,G,A,106,0.037736,0,"51,51,2,2",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,orf1ab,Gta/Ata,NC_045512:G:1895:A
3,SRR10903401,NC_045512,2407,G,T,123,0.024390,0,"57,63,1,2",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,orf1ab,aaG/aaT,NC_045512:G:2407:T
4,SRR10903401,NC_045512,3379,A,G,121,0.024793,0,"56,62,1,2",LOW,SILENT,SYNONYMOUS_CODING,orf1ab,gtA/gtG,NC_045512:A:3379:G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77116,SRR11410528,NC_045512,17934,C,T,1249,0.056045,7,"433,744,20,51",LOW,SILENT,SYNONYMOUS_CODING,orf1ab,acC/acT,NC_045512:C:17934:T
77117,SRR11410528,NC_045512,23403,A,G,1684,0.988717,0,"0,0,380,1304",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,S,gAt/gGt,NC_045512:A:23403:G
77118,SRR11410528,NC_045512,25563,G,T,679,0.986745,0,"0,0,130,549",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,ORF3a,caG/caT,NC_045512:G:25563:T
77119,SRR11410528,NC_045512,26844,T,A,993,0.008056,35,"235,750,7,1",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,M,Tcc/Acc,NC_045512:T:26844:A


# Split into high and low frequency variants table

Let's split up this table into ultra-low, low, and high frequency varaints (based on the `AF` column).

In [82]:
data_hf = data[data['AF'] > 0.3].copy()
data_lf = data[(data['AF'] >= 0.01) & (data['AF'] <= 0.3)].copy()
data_ulf = data[data['AF'] < 0.01].copy()
print(f"High frequency: {len(data_hf)}")
print(f"Low frequency: {len(data_lf)}")
print(f"Ultra-low frequency: {len(data_ulf)}")

High frequency: 160
Low frequency: 1825
Ultra-low frequency: 75136


We're going to first look at the low frequency variants.

# Define identifier of each transaction

The data is organized such that each genome name is an item, which are grouped into "transactions" based on the particular variant. We can group samples together based on shared SNVs (shared transaction IDs).

In [84]:
def group_transactions(df):
    return df.groupby('transaction_id').agg(
        {'Sample': lambda x: tuple(x)}).reset_index().set_index('transaction_id')

transaction_table_lf = group_transactions(data_lf)
transaction_table_lf.head(5)

,Sample
transaction_id,
NC_045512:A:10028:G,"(SRR11177792,)"
NC_045512:A:10042:G,"(SRR11177792,)"
NC_045512:A:10174:G,"(SRR11177792,)"
NC_045512:A:10181:T,"(SRR11177792,)"
NC_045512:A:10218:G,"(SRR11177792,)"


# Low frequency

Let's try some data mining on the low frequency variants

In [85]:
def mine(df, min_sup, min_conf):
    from efficient_apriori import apriori
    transactions = df['Sample'].values.tolist()
    
    return apriori(transactions, min_support=min_sup, min_confidence=min_conf)

itemsets, rules = mine(transaction_table_lf, 0.003, 1.0)
itemsets

{1: {('SRR11177792',): 141,
  ('SRR11059946',): 266,
  ('SRR11140750',): 103,
  ('SRR11059942',): 298,
  ('SRR10903401',): 69,
  ('SRR10903402',): 106,
  ('SRR11059943',): 73,
  ('SRR11059945',): 83,
  ('SRR11059944',): 118,
  ('SRR11059947',): 92,
  ('SRR10971381',): 88,
  ('SRR11140746',): 22,
  ('SRR11410540',): 34,
  ('SRR11397719',): 70,
  ('SRR11410532',): 19,
  ('SRR11410528',): 10,
  ('SRR11397728',): 17,
  ('SRR11140744',): 21,
  ('SRR11140748',): 25,
  ('SRR11410541',): 16,
  ('SRR11410536',): 12,
  ('SRR11410542',): 28,
  ('SRR11410529',): 11,
  ('SRR11397718',): 20,
  ('SRR11410533',): 6,
  ('SRR11410538',): 34,
  ('SRR11393704',): 10,
  ('SRR11409417',): 7,
  ('SRR11314339',): 7},
 2: {('SRR10903402', 'SRR11059944'): 20,
  ('SRR10903402', 'SRR11059947'): 16,
  ('SRR11059944', 'SRR11059947'): 38,
  ('SRR11059944', 'SRR11059946'): 22,
  ('SRR11059946', 'SRR11059947'): 21,
  ('SRR10903402', 'SRR11059946'): 9,
  ('SRR11059942', 'SRR11059946'): 19,
  ('SRR11140746', 'SRR1114074

Only a few rules. But then, we only had a few low-frequency variants.

# High frequency

Let's look at the high frequency

In [90]:
transaction_table_hf = group_transactions(data_hf)
itemsets, rules = mine(transaction_table_hf, 0.05, 1.0)
rules

[{SRR11410533} -> {SRR11397720},
 {SRR11410533} -> {SRR11410532},
 {SRR11410533} -> {SRR11410536},
 {SRR11410529} -> {SRR11410528},
 {SRR11410528} -> {SRR11410529},
 {SRR11410532, SRR11410533} -> {SRR11397720},
 {SRR11397720, SRR11410533} -> {SRR11410532},
 {SRR11397720, SRR11410532} -> {SRR11410533},
 {SRR11410533} -> {SRR11397720, SRR11410532},
 {SRR11410532, SRR11410536} -> {SRR11397720},
 {SRR11397720, SRR11410536} -> {SRR11410532},
 {SRR11397720, SRR11410532} -> {SRR11410536},
 {SRR11410533, SRR11410536} -> {SRR11397720},
 {SRR11397720, SRR11410536} -> {SRR11410533},
 {SRR11397720, SRR11410533} -> {SRR11410536},
 {SRR11410533} -> {SRR11397720, SRR11410536},
 {SRR11410533, SRR11410536} -> {SRR11410532},
 {SRR11410532, SRR11410536} -> {SRR11410533},
 {SRR11410532, SRR11410533} -> {SRR11410536},
 {SRR11410533} -> {SRR11410532, SRR11410536},
 {SRR11397714, SRR11410529} -> {SRR11410528},
 {SRR11397714, SRR11410528} -> {SRR11410529},
 {SRR11410532, SRR11410533, SRR11410536} -> {SRR11397

# Ultra-low frequency

Let's look at the majority of variants found, the ultra-low frequency ones (which I assume are likely sequencing errors?)

In [94]:
transaction_table_ulf = group_transactions(data_ulf)
itemsets, rules = mine(transaction_table_ulf, 0.05, 0.9)
rules

[{SRR11140748} -> {SRR11177792},
 {SRR11140744} -> {SRR11177792},
 {SRR11140746} -> {SRR11177792},
 {SRR11059946, SRR11177792} -> {SRR11059947},
 {SRR11059946, SRR11059947} -> {SRR11177792},
 {SRR11059947, SRR11140748} -> {SRR11177792},
 {SRR11059947, SRR11140744} -> {SRR11177792},
 {SRR11059947, SRR11140746} -> {SRR11177792}]